<a href="https://colab.research.google.com/github/MrHidr/AUT.CDM.matrix-accelerated-fpm/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 >**Project
information**  =>  **SN**: *404112060*  |  **SName**: *Mahdi Heydari*  |  CDM Course | Project 1 | Oct 2025

# A Survey of Matrix Monitoring Methods for Accelerating Frequent Pattern Mining Algorithms | First project of Computational Data Mining Course

## summary
[[will be added]]

# Step1: Fetching & Cleaning Data

### Installing Required Packages
ucimlrepo package : fetching dataset from UCI (UC Irvine Machine Learning Repository)

pandas : dataframe control


In [8]:
!pip install ucimlrepo

In [106]:
#importing packages
from ucimlrepo import fetch_ucirepo
import pandas as pd

### Fetching Online Retail Dataset from UCI
Documentation of UCI repo : https://github.com/uci-ml-repo/ucimlrepo

In [107]:
online_retail = fetch_ucirepo(id=352)

In [111]:
# data (as pandas dataframes)
X = online_retail.data.features
I = online_retail.data.ids
df = pd.concat([I, X], axis=1)
print("fetched dataset 5 rows as sample:\n", df.head(5))
print("\n","="*30,"\n")
df.info()
print("dataset info:\n", df.describe())

fetched dataset 5 rows as sample:
   InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       -----

In [112]:
initial_rows = df.shape[0]
print("Initial row count before any:", initial_rows)

Initial row count before any: 541909


In [113]:
# Filter for rows where Quantity is 0
zero_quantity_rows = df[df['Description'] == "returned"]

# Display the 'Description' and other useful columns for these rows
print(zero_quantity_rows[['Quantity', 'Description', 'UnitPrice', 'CustomerID']].head(77))

        Quantity Description  UnitPrice  CustomerID
166646         2    returned        0.0         NaN
166647         2    returned        0.0         NaN


### Cleaning Data; Removing Incomplete Records with Missing columns
According to dataset info, Some rows has missing data on "Description" and/or "CustomerID" Columns, as first part of cleaning, we will delete these rows.

In [114]:
df_cleaned = df.dropna(subset=['CustomerID', 'Description'])
rows_removed = initial_rows - df_cleaned.shape[0]

In [115]:
output_rows_count = df_cleaned.shape[0]
print("row count after removing rows: ", output_rows_count)
print("removed row count: ", initial_rows - output_rows_count)

row count after removing rows:  406829
removed row count:  135080


we had 4068290 rows with no CustomerID, which is equal to our rows after removing rows with no CustomerID and/or no Description so as result it means if a row has no Description it will have no CustomerID too.

###Cleaning Data; Negetive Values (& Returned Invoices)
While min value of 'Quantity' and 'UnitPrice' are -80995.000000 and -11062.060000, we have rows with negetive quantity and price which make these rows are invalid.
as mentioned in brief we should remove return invoices too, which is covered by removing rows with negetive "Quantity" and as more overthinking, in think of deleting متناظر negetive quantities, the goal of Not loosing data of selected baskets and not being a financial prefossional we just remove negetive baskets.

In [116]:
df_cleaned = df_cleaned[df_cleaned['Quantity'] > 0]
df_cleaned = df_cleaned[df_cleaned['UnitPrice'] >= 0]

In [117]:
cleaned_rows_count = df_cleaned.shape[0]
print("row count after removing rows with neg values: ", cleaned_rows_count)
print("removed row count (Neg Price and/or Neg Quantity): ", output_rows_count - cleaned_rows_count)

row count after removing rows with neg values:  397924
removed row count (Neg Price and/or Neg Quantity):  8905


# Step2: Item-Transaction Matrix

## Count Matrix
(rows='InvoiceNo',columns='StockCode count')

In [148]:
count_matrix = df_cleaned.pivot_table(index='InvoiceNo',
                                      columns='StockCode',
                                      aggfunc='size' ,
                                      fill_value=0)

In [149]:
print(count_matrix.head(10))
print("="*30)
print("number of unique InvoiceNo: ", count_matrix.shape[0] ,"\nnumber of unique StockCode", count_matrix.shape[1])

StockCode  10002  10080  10120  10123C  10124A  10124G  10125  10133  10135  \
InvoiceNo                                                                     
536365         0      0      0       0       0       0      0      0      0   
536366         0      0      0       0       0       0      0      0      0   
536367         0      0      0       0       0       0      0      0      0   
536368         0      0      0       0       0       0      0      0      0   
536369         0      0      0       0       0       0      0      0      0   
536370         1      0      0       0       0       0      0      0      0   
536371         0      0      0       0       0       0      0      0      0   
536372         0      0      0       0       0       0      0      0      0   
536373         0      0      0       0       0       0      0      0      0   
536374         0      0      0       0       0       0      0      0      0   

StockCode  11001  ...  90214V  90214W  90214Y  9021

## Binary Matrix

In [151]:
binary_matrix = (count_matrix > 0).astype(int)
print(binary_matrix.head(10))

StockCode  10002  10080  10120  10123C  10124A  10124G  10125  10133  10135  \
InvoiceNo                                                                     
536365         0      0      0       0       0       0      0      0      0   
536366         0      0      0       0       0       0      0      0      0   
536367         0      0      0       0       0       0      0      0      0   
536368         0      0      0       0       0       0      0      0      0   
536369         0      0      0       0       0       0      0      0      0   
536370         1      0      0       0       0       0      0      0      0   
536371         0      0      0       0       0       0      0      0      0   
536372         0      0      0       0       0       0      0      0      0   
536373         0      0      0       0       0       0      0      0      0   
536374         0      0      0       0       0       0      0      0      0   

StockCode  11001  ...  90214V  90214W  90214Y  9021

as we see each row stands for one InvoiceNo and each column show apearance of stock in InvoiceNo (Basket).

In [152]:
#accessing row data by InvoiceID

# Step3: Streaming Simulation